# 探索 ONNX 模型输入输出

本 Notebook 用于逐步加载 ONNX 模型，并检查其输入和输出的详细信息，以便正确准备推理所需的数据。

In [1]:
import openvino.runtime as ov
from pathlib import Path
import numpy as np

c:\Users\k\.conda\envs\openvino\lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


## 1. 配置模型路径和推理设备

In [2]:
# --- 配置 ---
# 输入：你的 ONNX 模型文件路径
# 你可以更改为列表中的其他 .onnx 文件，例如 model_fp16.onnx
onnx_model_path = r"C:\\Users\\k\\Desktop\\BaiduSyncdisk\\baidu_sync_documents\\hf_models\\jina-clip-v2\\onnx\\model.onnx"

# 推理设备，例如 "CPU", "GPU", "NPU"
device_name = "CPU"

model_file = Path(onnx_model_path)
if not model_file.exists():
    print(f"错误：ONNX 模型文件 '{onnx_model_path}' 不存在。请检查路径。")
else:
    print(f"ONNX 模型文件路径: {onnx_model_path}")
    print(f"推理设备: {device_name}")

ONNX 模型文件路径: C:\\Users\\k\\Desktop\\BaiduSyncdisk\\baidu_sync_documents\\hf_models\\jina-clip-v2\\onnx\\model.onnx
推理设备: CPU


## 2. 初始化 OpenVINO Core 并读取模型

In [3]:
core = ov.Core()
print(f"可用的 OpenVINO 设备: {core.available_devices}")

model = None
if model_file.exists():
    try:
        print(f"正在从 '{onnx_model_path}' 读取 ONNX 模型...")
        model = core.read_model(model=str(model_file)) # Path 对象需要转换为 str
        print("模型读取成功。")
    except Exception as e:
        print(f"读取模型时发生错误: {e}")
else:
    print("模型文件未找到，跳过读取。")

可用的 OpenVINO 设备: ['CPU', 'GPU', 'NPU']
正在从 'C:\\Users\\k\\Desktop\\BaiduSyncdisk\\baidu_sync_documents\\hf_models\\jina-clip-v2\\onnx\\model.onnx' 读取 ONNX 模型...
模型读取成功。


## 3. 检查模型输入信息

In [4]:
if model:
    print("\n--- 模型输入信息 ---")
    for i, input_tensor in enumerate(model.inputs):
        print(f"  输入 #{i}:")
        print(f"    名称 (any_name): {input_tensor.any_name}")
        # 打印所有可能的名称
        # print(f"    所有名称: {input_tensor.names}") 
        print(f"    形状 (partial_shape): {input_tensor.partial_shape}")
        print(f"    数据类型: {input_tensor.element_type}")
        print(f"    Tensor 名称 (get_tensor().name): {input_tensor.get_tensor().name if hasattr(input_tensor.get_tensor(), 'name') else 'N/A'}")

else:
    print("模型未加载，无法显示输入信息。")


--- 模型输入信息 ---
  输入 #0:
    名称 (any_name): input_ids
    形状 (partial_shape): [?,?]
    数据类型: <Type: 'int64_t'>
    Tensor 名称 (get_tensor().name): N/A
  输入 #1:
    名称 (any_name): pixel_values
    形状 (partial_shape): [?,3,512,512]
    数据类型: <Type: 'float32'>
    Tensor 名称 (get_tensor().name): N/A


## 3.1 模型输入详细解析

模型具有两个输入:

1. **输入 #0 (input_ids)**:
  - 这是一个文本输入张量，用于接收文本标记ID
  - 形状为动态的 `[?,?]`，表示批次大小和序列长度都是可变的
  - 数据类型为64位整数 (`int64_t`)
  - 通常用于传递文本的token ID序列

2. **输入 #1 (pixel_values)**:
  - 这是一个图像输入张量，用于接收预处理后的图像数据
  - 形状为 `[?,3,512,512]`，表示:
    - 批次大小是动态的 (`?`)
    - 3个颜色通道 (RGB)
    - 图像尺寸为512×512像素
  - 数据类型为32位浮点数 (`float32`)
  - 通常用于传递归一化后的图像像素值

这是一个典型的多模态CLIP模型输入格式，同时处理文本和图像数据，用于计算文本和图像之间的相似度。

## 4. 检查模型输出信息

In [5]:
if model:
    print("\n--- 模型输出信息 ---")
    for i, output_tensor in enumerate(model.outputs):
        print(f"  输出 #{i}:")
        print(f"    名称 (any_name): {output_tensor.any_name}")
        # print(f"    所有名称: {output_tensor.names}")
        print(f"    形状 (partial_shape): {output_tensor.partial_shape}")
        print(f"    数据类型: {output_tensor.element_type}")
        print(f"    Tensor 名称 (get_tensor().name): {output_tensor.get_tensor().name if hasattr(output_tensor.get_tensor(), 'name') else 'N/A'}")

else:
    print("模型未加载，无法显示输出信息。")


--- 模型输出信息 ---
  输出 #0:
    名称 (any_name): text_embeddings
    形状 (partial_shape): [?,1024]
    数据类型: <Type: 'float32'>
    Tensor 名称 (get_tensor().name): N/A
  输出 #1:
    名称 (any_name): image_embeddings
    形状 (partial_shape): [?,1024]
    数据类型: <Type: 'float32'>
    Tensor 名称 (get_tensor().name): N/A
  输出 #2:
    名称 (any_name): l2norm_text_embeddings
    形状 (partial_shape): [?,1024]
    数据类型: <Type: 'float32'>
    Tensor 名称 (get_tensor().name): N/A
  输出 #3:
    名称 (any_name): l2norm_image_embeddings
    形状 (partial_shape): [?,1024]
    数据类型: <Type: 'float32'>
    Tensor 名称 (get_tensor().name): N/A


## 4.1 模型输出详细解析

模型共有四个输出：

1. **text_embeddings**:
    - 形状为 `[?,1024]`，表示批次大小为动态，每个文本的嵌入向量维度为1024
    - 数据类型为32位浮点数（`float32`）
    - 这是文本的原始嵌入向量表示

2. **image_embeddings**:
    - 形状为 `[?,1024]`，表示批次大小为动态，每个图像的嵌入向量维度为1024
    - 数据类型为32位浮点数（`float32`）
    - 这是图像的原始嵌入向量表示

3. **l2norm_text_embeddings**:
    - 形状为 `[?,1024]`，与text_embeddings相同
    - 数据类型为32位浮点数（`float32`）
    - 这是经过L2归一化处理后的文本嵌入向量，用于计算余弦相似度

4. **l2norm_image_embeddings**:
    - 形状为 `[?,1024]`，与image_embeddings相同
    - 数据类型为32位浮点数（`float32`）
    - 这是经过L2归一化处理后的图像嵌入向量，用于计算余弦相似度

L2归一化后的嵌入向量可直接用于计算文本与图像之间的余弦相似度，主要用于跨模态匹配任务（如图文检索）。

## 5. 准备输入数据 (需要根据上面的信息修改)

**重要:** 下面的代码块用于准备输入数据。你需要仔细查看上面单元格打印出的“模型输入信息”，并相应地修改此部分。
特别是要注意：
- **输入名称**: 确保 `inputs` 字典中的键与模型期望的输入名称完全匹配。
- **形状**: 根据 `partial_shape` 创建具有正确维度的 NumPy 数组。如果存在动态维度 (例如 `?` 或 `-1`)，你需要为其选择一个具体的值 (例如，批处理大小为1，序列长度根据你的数据决定)。
- **数据类型**: 确保 NumPy 数组的数据类型 (`dtype`) 与模型期望的 `element_type` 一致。

In [6]:
inputs = {}
if model:
    print("请根据以下模型输入信息，修改下面的虚拟数据准备代码：")
    for input_tensor in model.inputs:
        print(f"  - 输入名称: {input_tensor.any_name}, 形状: {input_tensor.partial_shape}, 数据类型: {input_tensor.element_type}")

    # --- !!! 开始修改此部分 !!! ---
    # 根据用户提供的模型输入信息准备数据
    
    # 输入1: input_ids
    input_name_1 = "input_ids" 
    # 形状 [batch_size, sequence_length], 例如 [1, 77]
    shape_1 = [1, 77] 
    dtype_1 = np.int64 
    # 假设 token IDs 范围在 0 到模型词汇表大小减1 (例如 30000 for many tokenizers)
    dummy_data_1 = np.random.randint(0, 30000, size=shape_1, dtype=dtype_1)
    inputs[input_name_1] = dummy_data_1
    print(f"为输入 '{input_name_1}' 创建了形状为 {shape_1}，类型为 {dtype_1} 的虚拟数据。")

    # 输入2: pixel_values
    input_name_2 = "pixel_values" 
    # 形状 [batch_size, channels, height, width], 例如 [1, 3, 512, 512]
    shape_2 = [1, 3, 512, 512] 
    dtype_2 = np.float32 
    # 像素值通常在 [0, 1] 或 [-1, 1] 范围内，具体取决于预处理
    # 这里我们生成随机的 float32 数据，模拟归一化后的像素值 (例如，在0到1之间)
    dummy_data_2 = np.random.rand(*shape_2).astype(dtype_2)
    inputs[input_name_2] = dummy_data_2
    print(f"为输入 '{input_name_2}' 创建了形状为 {shape_2}，类型为 {dtype_2} 的虚拟数据。")
    # --- !!! 结束修改此部分 !!! ---

    if not inputs:
        print("\\n错误：未能准备任何输入数据。请检查上面的模型输入信息并修改此单元格中的代码。")
else:
    print("模型未加载，无法准备输入数据。")

请根据以下模型输入信息，修改下面的虚拟数据准备代码：
  - 输入名称: input_ids, 形状: [?,?], 数据类型: <Type: 'int64_t'>
  - 输入名称: pixel_values, 形状: [?,3,512,512], 数据类型: <Type: 'float32'>
为输入 'input_ids' 创建了形状为 [1, 77]，类型为 <class 'numpy.int64'> 的虚拟数据。
为输入 'pixel_values' 创建了形状为 [1, 3, 512, 512]，类型为 <class 'numpy.float32'> 的虚拟数据。


## 6. 编译模型并执行推理

In [7]:
compiled_model = None
infer_request = None
results = None

if model and inputs:
    try:
        print(f"\\n正在将模型编译到设备 '{device_name}'...")
        compiled_model = core.compile_model(model=model, device_name=device_name)
        print("模型编译成功。")

        infer_request = compiled_model.create_infer_request()
        print("推理请求创建成功。")
        
        print(f"\\n正在使用准备好的输入进行推理...")
        # print("输入数据详情:")
        # for name, data in inputs.items():
        #    print(f"  {name}: shape={data.shape}, dtype={data.dtype}")

        results = infer_request.infer(inputs=inputs)
        print("推理完成。")

    except Exception as e:
        print(f"\\n编译或推理过程中发生错误: {e}")
        print("请检查：")
        print("1. 输入数据是否与模型期望的完全一致（名称、形状、数据类型）。")
        print(f"2. 设备 '{device_name}' 是否可用且模型是否支持在该设备上运行。")

elif not model:
    print("模型未加载，无法编译或推理。")
else:
    print("输入数据未准备好，无法编译或推理。")

\n正在将模型编译到设备 'CPU'...
模型编译成功。
推理请求创建成功。
\n正在使用准备好的输入进行推理...
推理完成。


## 7. 查看推理结果

In [ ]:
if results:
    print("\\n--- 推理结果 ---")
    # `results` 是一个字典，键是输出张量的名称 (通常是 compiled_model.outputs[i].get_tensor().name)
    # 值是包含数据的 NumPy 数组。
    
    # 打印 compiled_model 的输出张量信息，以便与 results 字典的键进行匹配
    # print("\\nCompiled Model Output Tensor Info:")
    # for i, out_tensor_node in enumerate(compiled_model.outputs):
    #     print(f"  Output Node #{i}:")
    #     print(f"    Any Name: {out_tensor_node.get_any_name()}") # 通常是原始模型中的名称
    #     print(f"    Tensor Names: {out_tensor_node.get_names()}") # 可能包含多个名称，包括内部名称
    #     # print(f"    Tensor Name (get_tensor().name): {out_tensor_node.get_tensor().name}") # 这是结果字典中常用的键

    print("\\nResults Dictionary:")
    for output_name_from_result, result_data in results.items():
        print(f"  输出名称 (from results key): {output_name_from_result}")
        print(f"    形状: {result_data.shape}")
        print(f"    数据类型: {result_data.dtype}")
        print(f"    部分数据: {result_data.flatten()[:10]}...") # 打印前10个扁平化数据
        
    # 你也可以尝试通过 compiled_model.outputs 来迭代并获取结果
    # print("\\nIterating through compiled_model.outputs:")
    # for output_node in compiled_model.outputs:
    #     try:
    #         # 尝试使用 output_node 直接从 results 中获取数据
    #         # 这依赖于 OpenVINO 如何将 output_node 映射到 results 字典的键
    #         # 最可靠的方式通常是使用 output_node.get_tensor().name 或 output_node.get_any_name()
    #         # 如果这些名称与 results 字典中的键不完全匹配，你可能需要更复杂的匹配逻辑
            
    #         # 尝试使用 any_name (通常是原始模型中的名称)
    #         key_to_try = output_node.get_any_name()
    #         if key_to_try in results:
    #             result_data = results[key_to_try]
    #             print(f"  输出名称 (any_name): {key_to_try}")
    #             print(f"    形状: {result_data.shape}")
    #             print(f"    部分数据: {result_data.flatten()[:10]}...")
    #         else:
    #             # 尝试使用 tensor name (通常是结果字典中的键)
    #             # 有时 get_tensor().name 可能不存在或不唯一
    #             found_by_tensor_name = False
    #             if hasattr(output_node.get_tensor(), 'name'):
    #                 tensor_name = output_node.get_tensor().name
    #                 if tensor_name in results:
    #                     result_data = results[tensor_name]
    #                     print(f"  输出名称 (tensor_name): {tensor_name}")
    #                     print(f"    形状: {result_data.shape}")
    #                     print(f"    部分数据: {result_data.flatten()[:10]}...")
    #                     found_by_tensor_name = True
                
    #             if not found_by_tensor_name:
    #                 print(f"  警告: 未能在 results 中找到输出 '{key_to_try}' 或其 tensor name。")
    #                 print(f"    Available keys in results: {list(results.keys())}")

    #     except Exception as e_res:
    #         print(f"处理输出 {output_node.get_any_name()} 时出错: {e_res}")
            
elif infer_request and not results:
    print("推理已执行，但 results 为空。可能在推理步骤中发生了错误，或者模型没有输出。")
elif model and not inputs:
    print("输入数据未准备，无法查看结果。")
else:
    print("模型未加载或推理未执行，无法显示结果。")

## 8. (可选) 清理资源
虽然 Python 的垃圾回收通常会处理，但在某些情况下，显式删除对象可能有助于更快地释放资源，尤其是在处理大型模型或在循环中操作时。
对于 OpenVINO 对象，通常不需要手动删除，因为它们会在超出作用域时被清理。

In [ ]:
# del model
# del compiled_model
# del infer_request
# print("资源已清理 (如果取消注释)")